In [1]:
import scanpy as sc
import pandas as pd
import os
from dca.api import dca
import tensorflow as tf
from tensorflow.python.keras.backend import set_session

2021-07-30 11:22:23.903062: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-30 11:22:23.903078: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/kopt/config.py:60: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _config = yaml.load(open(_config_path))


In [2]:
sample = 'Sample5_80_percent'

Loading the read count data and potentially removing cell doublets/mulitplets.

In [3]:
dat = pd.read_table('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/tsv/' + sample + '.barcode.cell.distribution.tsv',sep='\t')
remove_mixed = True
if remove_mixed:
    clust_file = pd.read_csv('/users/lvelten/project/Methylome/analysis/missionbio/tapestri/'+ sample + '/tsv/cluster_assignment.tsv',
                            sep='\t')
    drop_rows = clust_file['Barcode'][clust_file['CellType']=='Mixed']
    dat = dat.drop(drop_rows)

Creating an AnnData object from the read counts.

In [4]:
adata = sc.AnnData(dat)
sc.pp.filter_genes(adata,min_cells=1)

Running the autoencoder: Currently, we have the following available architectures:

    -'meth-encoder': A classical autoencoder, where the dispersion parameters of both NB-distributions can be freely selected by the model.
    -'meth-encoder-constant': An autoencoder, where the dispersion parameters is fixed for each gene. This lowers the number of paramters substantially.
    -'meth-encoder-poisson': An autoencoder, which mixes a negative bionomial distribution (foreground) with a Poisson distribution as the background
    -'meth-encoder-poisson-constant': Same as the above, but with a fixed dispersion for each gene

In [5]:
res = dca(adata,
          ae_type='meth-simple-encoder',
          return_info=True,
          return_model=True,
          return_bottleneck=True,
          verbose=True,
          early_stop=50,
          network_kwds={'debug': True},
          init='glorot_normal',
         hidden_size=(16,8,16))

2021-07-30 11:22:26.717235: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-30 11:22:26.717382: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-30 11:22:26.717395: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-30 11:22:26.717410: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CZC041BTPK): /proc/driver/nvidia/version does not exist


dca: Successfully preprocessed 199 genes and 5207 cells.



2021-07-30 11:22:26,871 [WARNING] From /users/lvelten/project/Methylome/src/error_mod/dca/dca/train.py:41: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

2021-07-30 11:22:26.873307: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              [(None, 199)]        0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 16)           3200        count[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16)           48          enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 16)           0           batch_normalization[0][0]        
____________________________________________________________________________________________

2021-07-30 11:22:27.681601: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
2021-07-30 11:22:27.704049: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


4686/4686 [==============================] - 1s 139us/sample - loss: 80.3345 - val_loss: 81.1404
Epoch 2/300
2272/4686 [=============>................] - ETA: 0s - loss: 79.4717

/users/mscherer/software/anaconda3/envs/dca/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


4686/4686 [==============================] - 0s 48us/sample - loss: 79.8257 - val_loss: 80.7826
Epoch 3/300
4686/4686 [==============================] - 0s 47us/sample - loss: 79.6470 - val_loss: 80.6803
Epoch 4/300
4686/4686 [==============================] - 0s 47us/sample - loss: 79.6032 - val_loss: 80.6545
Epoch 5/300
4686/4686 [==============================] - 0s 47us/sample - loss: 79.5904 - val_loss: 80.6465
Epoch 6/300
4686/4686 [==============================] - 0s 49us/sample - loss: 79.5862 - val_loss: 80.6425
Epoch 7/300
4686/4686 [==============================] - 0s 49us/sample - loss: 79.5830 - val_loss: 80.6394
Epoch 8/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5805 - val_loss: 80.6365
Epoch 9/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5787 - val_loss: 80.6343
Epoch 10/300
4686/4686 [==============================] - 0s 46us/sample - loss: 79.5769 - val_loss: 80.6336
Epoch 11/300
4686/4686 [===============

4686/4686 [==============================] - 0s 49us/sample - loss: 79.5574 - val_loss: 80.6203
Epoch 74/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5579 - val_loss: 80.6202
Epoch 75/300
4686/4686 [==============================] - 0s 49us/sample - loss: 79.5586 - val_loss: 80.6203
Epoch 76/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5575 - val_loss: 80.6203
Epoch 77/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5584 - val_loss: 80.6202
Epoch 78/300
4686/4686 [==============================] - 0s 49us/sample - loss: 79.5587 - val_loss: 80.6203
Epoch 79/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5580 - val_loss: 80.6203
Epoch 80/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5584 - val_loss: 80.6203
Epoch 81/300
4686/4686 [==============================] - 0s 48us/sample - loss: 79.5584 - val_loss: 80.6203
Epoch 82/300
4448/4686 [========

Writing the data on disk.

In [6]:
di = '/users/lvelten/project/Methylome/analysis/missionbio/tapestri/' + sample + '/methylation_autoencoder/'
if not os.path.isdir(di):
    os.mkdir(di)
    
pd.DataFrame(adata.obsm['X_meth_value']).to_csv(di + 'mixture_prob_dca.csv')

In [7]:
foo = res.get_encoder().predict([adata.X, adata.X.mean(1)])

In [8]:
pd.DataFrame(adata.obsm['X_bottleneck']).to_csv(di + 'bottleneck.csv')

In [9]:
pd.DataFrame(adata.obsm['lambda_poisson']).to_csv(di + 'lambda_poisson_neg_binom.csv')

KeyError: 'lambda_poisson'

In [ ]:
pd.DataFrame(adata.var['meth_dispersion_nb']).to_csv(di + 'dispersion_nb_neg_binom.csv')

In [ ]:
adata.obsm['X_meth_value']